# Equal-Weight S&P 500 Index Fund
#### The Standard and Poor's 500 or S&P 500 is the most famous financial benchmark in the world.This stock market index tracks the performance of 500 large companies listed on stock exchanges in the United States. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.


## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import requests 
import math
from secret1 import IEX_CLOUD_API_TOKEN

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Importing list of stocks in S&P 500 index
#### The constituents of this index change over time, so in an ideal world we would connect directly to the index provider and pull their real-time constituents on a regular basis. But,here I'm using a static version of the S&P 500 constituents (for the purpose of this project) available on web for free & moving this file into the project directory so it can be accessed by other files in the directory.

In [3]:
stocks = pd.read_csv('sp_500_stocks (1).csv')
stocks[:5]

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


## Acquiring an API Token
#### Now we need an API Token from one of the data providers to get the financial data. Here I'm using IEX CLOUD API TOKEN and stored my API Token in secret1.py file that is available in project directory.


In [4]:
symbol='aap'
api_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol}?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()[0]
data

{'avgTotalVolume': 1392022,
 'calculationPrice': 'close',
 'change': -1.45,
 'changePercent': -0.02094,
 'close': 67.79,
 'closeSource': 'official',
 'closeTime': 1706907602399,
 'companyName': 'Advance Auto Parts Inc',
 'currency': 'USD',
 'delayedPrice': 67.8,
 'delayedPriceTime': 1706907573853,
 'extendedChange': 0.33,
 'extendedChangePercent': 0.00487,
 'extendedPrice': 68.12,
 'extendedPriceTime': 1706921738015,
 'high': 68.85,
 'highSource': '15 minute delayed price',
 'highTime': 1706907599977,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 67.79,
 'iexCloseTime': 1706907592308,
 'iexLastUpdated': 1706907592308,
 'iexMarketPercent': 0.04277610527902999,
 'iexOpen': 68.35,
 'iexOpenTime': 1706884224470,
 'iexRealtimePrice': 67.79,
 'iexRealtimeSize': 67,
 'iexVolume': 57589,
 'lastTradeTime': 1706907599977,
 'latestPrice': 67.79,
 'latestSource': 'Close',
 'latestTime': 'February 2, 2024',
 'latestUpdate': 1706907602399,
 'latestVolume': 13

## Adding the Stocks Data to a Pandas DataFrame


In [5]:
my_columns = ['Ticker', 'Stock Price ($)', 'Market Cap (BILLION USD$)', 'No. of shares to buy']
df = pd.DataFrame(columns=my_columns)
df

,Ticker,Stock Price ($),Market Cap (BILLION USD$),No. of shares to buy


In [6]:
my_columns = ['Ticker', 'Stock Price ($)', 'Market Cap (BILLION USD$)', 'No. of shares to buy']
df = pd.DataFrame(columns=my_columns)

new_data = {'Ticker':data['symbol'], 
            'Stock Price ($)':data['latestPrice'], 
            'Market Cap (BILLION USD$)':data['marketCap']/1000000000, 
            'No. of shares to buy':['NA']}

new_df = pd.DataFrame(new_data)

result_df = pd.concat([df, new_df], ignore_index=True)

print(result_df)

  Ticker  Stock Price ($)  Market Cap (BILLION USD$) No. of shares to buy
0    AAP            67.79                    4.03364                   NA


## Looping Through The Tickers in Our List of Stocks

In [7]:
final_df = pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker'][:5]:
    api_url = f'https://api.iex.cloud/v1/data/core/quote/{stock}?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()[0]
    new_data = {'Ticker':data['symbol'], 
            'Stock Price ($)':data['latestPrice'], 
            'Market Cap (BILLION USD$)':data['marketCap']/1000000000, 
            'No. of shares to buy':['NA']}

    new_df = pd.DataFrame(new_data)

    final_df = pd.concat([final_df, new_df], ignore_index=True)


In [8]:
final_df

,Ticker,Stock Price ($),Market Cap (BILLION USD$),No. of shares to buy
0,A,132.97,38.960755,NA
1,AAL,14.56,9.515550,NA
2,AAP,67.79,4.033640,NA
3,AAPL,185.85,2873.593372,NA
4,ABBV,168.67,303.606000,NA


## Using Batch API Calls to Improve Performance

In [9]:
# Here I'm defining a function that generates evenly sized chunks:

def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [10]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    # print(symbol_strings[i])
my_columns2 = ['Ticker','Name', 'Stock Price ($)', 'Market Cap (BILLION USD$)', 'No. of shares to buy']

final_df = pd.DataFrame(columns=my_columns2)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        if symbol in data:
            new_data = {'Ticker':symbol,
                        'Name':data[symbol]['quote']['companyName'],
                        'Stock Price ($)':data[symbol]['quote']['latestPrice'], 
                        'Market Cap (BILLION USD$)': data[symbol]['quote']['marketCap'],
                        'No. of shares to buy':['NA']}
            new_df = pd.DataFrame(new_data)
            new_df['Market Cap (BILLION USD$)'] = new_df['Market Cap (BILLION USD$)'].fillna(0)/1000000000


            final_df = pd.concat([final_df, new_df], ignore_index=True)

final_df

,Ticker,Name,Stock Price ($),Market Cap (BILLION USD$),No. of shares to buy
0,A,Agilent Technologies Inc.,132.97,38.960755,NA
1,AAL,American Airlines Group Inc,14.56,9.515550,NA
2,AAP,Advance Auto Parts Inc,67.79,4.033640,NA
3,AAPL,Apple Inc,185.85,2873.593372,NA
4,ABBV,Abbvie Inc,168.67,303.606000,NA
...,...,...,...,...,...
491,YUM,Yum Brands Inc.,128.75,36.089683,NA
492,ZBH,Zimmer Biomet Holdings Inc,126.32,26.398443,NA
493,ZBRA,Zebra Technologies Corp. - Class A,246.48,12.659115,NA
494,ZION,Zions Bancorporation N.A,39.65,5.874100,NA


## Calculating the no. of shares to buy

In [11]:
portfoilo_size = input('Please enter the value of portfolio: ')
try:
    val = float(portfoilo_size)
except:
    print('Please enter a valid amount')
    portfoilo_size = input('Please enter the value of portfolio: ')

portfoilo_size

Please enter the value of portfolio:  1000000


'1000000'

In [12]:
position_size = val/len(final_df.index)

for i in range(0, len(final_df.index)):
    final_df.loc[i, 'No. of shares to buy'] = math.floor(position_size/final_df.loc[i, 'Stock Price ($)'])

    # final_df.loc[i, 'No. of shares to buy'] == math.floor(position_size/final_df.loc[i, 'Stock Price'])

final_df

,Ticker,Name,Stock Price ($),Market Cap (BILLION USD$),No. of shares to buy
0,A,Agilent Technologies Inc.,132.97,38.960755,15
1,AAL,American Airlines Group Inc,14.56,9.515550,138
2,AAP,Advance Auto Parts Inc,67.79,4.033640,29
3,AAPL,Apple Inc,185.85,2873.593372,10
4,ABBV,Abbvie Inc,168.67,303.606000,11
...,...,...,...,...,...
491,YUM,Yum Brands Inc.,128.75,36.089683,15
492,ZBH,Zimmer Biomet Holdings Inc,126.32,26.398443,15
493,ZBRA,Zebra Technologies Corp. - Class A,246.48,12.659115,8
494,ZION,Zions Bancorporation N.A,39.65,5.874100,50


## Saving Our Pandas DataFrame to Excel Output

In [13]:
excel_file_path = 'my_fund.xlsx'

# Save the DataFrame to Excel
final_df.to_excel(excel_file_path, index=False)